In [1]:
import  pandas  as pd
import numpy as np
import sys
import os

pos="datas/1/"
neg="datas/0/"

pos_files=[]
pos_databases=[]
def process(data):
    features=data.columns[2:]
    a=data[features[0]].count()
    data=data[features]
    data['index']=range(a)
    data.set_index('index')
    while(a<15):
        data.loc[-a]=0
        a+=1
    data['index']=range(a)
    if(a>15):
        data=data.head(15)
    data.set_index('index')
    temp_data = np.array(data[features],float).flatten().tolist()
    for i in range(len(temp_data)):
        if(temp_data[i]>0):
            temp_data[i]=1
    return temp_data
            
for root, dirs, files in os.walk(pos):
    pos_files=files
for root, dirs, files in os.walk(neg):
    neg_files=files    
test=[]
label=[]
for path in pos_files:
    df=pd.read_excel(pos+path,sheet_name='sheet2')
    pos_databases=df.groupby("数据集名称").count().index.tolist()
    print(pos_databases)
    data=df.dropna()
    for database in pos_databases:
        model_databases=data[data["数据集名称"]==database]
        resuttemp=process(model_databases)
        for i in range(83):
            test.append(resuttemp)
            label.append([1,0])

for path in neg_files:
    df=pd.read_excel(neg+path,sheet_name='sheet2')
    neg_databases=df.groupby("数据集名称").count().index.tolist()
    print(neg_databases)
    data=df.dropna()
    for database in neg_databases:
        model_databases=data[data["数据集名称"]==database]
        test.append(process(model_databases))
        label.append([0,1])
        
data_x=np.array(test,float)
data_y=np.array(label,float)

pos_databases

['email1b', 'facebook_audio2a', 'ftps_up_2b', 'scpDown1', 'sftp_up_2a', 'skype_chat1a']


c:\users\qcm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
c:\users\qcm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\qcm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

['demo0', 'demo1', 'demo10', 'demo100', 'demo101', 'demo102', 'demo103', 'demo104', 'demo105', 'demo106', 'demo107', 'demo108', 'demo109', 'demo11', 'demo110', 'demo111', 'demo112', 'demo113', 'demo114', 'demo115', 'demo116', 'demo117', 'demo118', 'demo119', 'demo12', 'demo120', 'demo121', 'demo122', 'demo123', 'demo124', 'demo125', 'demo126', 'demo127', 'demo128', 'demo129', 'demo13', 'demo130', 'demo131', 'demo132', 'demo133', 'demo134', 'demo135', 'demo136', 'demo137', 'demo138', 'demo139', 'demo14', 'demo140', 'demo141', 'demo142', 'demo143', 'demo144', 'demo145', 'demo146', 'demo147', 'demo148', 'demo149', 'demo15', 'demo150', 'demo151']
['demo10', 'demo100', 'demo101', 'demo102', 'demo103', 'demo104', 'demo105', 'demo106', 'demo107', 'demo108', 'demo109', 'demo11', 'demo110', 'demo111', 'demo112', 'demo113', 'demo114', 'demo115', 'demo116', 'demo117', 'demo118', 'demo119', 'demo12', 'demo120', 'demo121', 'demo122', 'demo123', 'demo124', 'demo125', 'demo126', 'demo127', 'demo128',

['email1b',
 'facebook_audio2a',
 'ftps_up_2b',
 'scpDown1',
 'sftp_up_2a',
 'skype_chat1a']

In [2]:
from lstm_cnn import LSTM_CNN   #OPTION 1
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

def gen_batch(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
# Model Hyperparameters
embedding_dim  = 32     #128
max_seq_legth = 70 
filter_sizes = [3,4,5]  #3
num_filters = 32
dropout_prob = 0.5 #0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size = 50
num_epochs = 40 
evaluate_every = 100 #100
checkpoint_every = 100000 #100
num_checkpoints = 0 #Checkpoints to store
max_document_length=15

# Misc Parameters
allow_soft_placement = True
log_device_placement = False
dev_size = .10
# Split train/test set

indices = np.random.permutation(data_x.shape[0])

x_shuffled = data_x[indices]
y_shuffled = data_y[indices]
dev_sample_index = -1 * int(dev_size * float(data_x.shape[0]))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    
    with sess.as_default():
        
        model = LSTM_CNN(x_train.shape[1],y_train.shape[1],2+1,embedding_dim,filter_sizes,num_filters,l2_reg_lambda)


        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(model.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", model.loss)
        acc_summary = tf.summary.scalar("accuracy", model.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        #TRAINING STEP
        def train_step(x_batch, y_batch,save=False):
            feed_dict = {
              model.input_x: x_batch,
              model.input_y: y_batch,
              model.dropout_keep_prob: dropout_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, model.loss, model.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if save:
                train_summary_writer.add_summary(summaries, step)
        #EVALUATE MODEL
        def dev_step(x_batch, y_batch, writer=None,save=False):
            feed_dict = {
              model.input_x: x_batch,
              model.input_y: y_batch,
              model.dropout_keep_prob: 0.5
            }
            
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, model.loss, model.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if save:
                if writer:
                    writer.add_summary(summaries, step)

        #CREATE THE BATCHES GENERATOR
        batches = gen_batch(list(zip(x_train, y_train)), batch_size, num_epochs)
        
        #TRAIN FOR EACH BATCH
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
        dev_step(x_dev, y_dev, writer=dev_summary_writer)

W0912 09:31:46.356950 18168 deprecation.py:323] From <ipython-input-2-ffbc56158245>:62: VocabularyProcessor.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.
W0912 09:31:46.358946 18168 deprecation.py:323] From c:\users\qcm\appdata\local\programs\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\preprocessing\text.py:154: CategoricalVocabulary.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.categorical_vocabulary) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.
W0912 09:31:46.358946 18168 deprecation_wrapper.py:119] From C:\Users\QCM\lstm_cnn.py:9: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 09:31:46.458678 18168 deprecation_wrapper.py:119] From C:\Users\QCM\lstm_cnn.py:18: Th

(!!) LOADED LSTM-CNN! :)
Writing to C:\Users\QCM\runs\1568251910

2019-09-12T09:31:53.353226: step 1, loss 0.797652, acc 0.56
2019-09-12T09:31:53.542719: step 2, loss 0.75815, acc 0.42
2019-09-12T09:31:53.689327: step 3, loss 0.767697, acc 0.48
2019-09-12T09:31:53.823966: step 4, loss 0.739375, acc 0.54
2019-09-12T09:31:53.966585: step 5, loss 0.805348, acc 0.42
2019-09-12T09:31:54.097234: step 6, loss 0.713212, acc 0.5
2019-09-12T09:31:54.232871: step 7, loss 0.702744, acc 0.54
2019-09-12T09:31:54.362524: step 8, loss 0.771263, acc 0.52
2019-09-12T09:31:54.496166: step 9, loss 0.678268, acc 0.54
2019-09-12T09:31:54.630807: step 10, loss 0.767325, acc 0.44
2019-09-12T09:31:54.791377: step 11, loss 0.673978, acc 0.6
2019-09-12T09:31:54.938981: step 12, loss 0.662402, acc 0.62
2019-09-12T09:31:55.096560: step 13, loss 0.723714, acc 0.44
2019-09-12T09:31:55.244165: step 14, loss 0.705458, acc 0.56
2019-09-12T09:31:55.400746: step 15, loss 0.645836, acc 0.6
2019-09-12T09:31:55.543364: step

2019-09-12T09:32:15.353329: step 133, loss 0.484403, acc 0.771429
2019-09-12T09:32:15.531344: step 134, loss 0.571287, acc 0.76
2019-09-12T09:32:15.701333: step 135, loss 0.569758, acc 0.66
2019-09-12T09:32:15.851354: step 136, loss 0.509352, acc 0.68
2019-09-12T09:32:16.024335: step 137, loss 0.500218, acc 0.76
2019-09-12T09:32:16.186866: step 138, loss 0.432107, acc 0.8
2019-09-12T09:32:16.367401: step 139, loss 0.378396, acc 0.84
2019-09-12T09:32:16.538381: step 140, loss 0.49776, acc 0.68
2019-09-12T09:32:16.708357: step 141, loss 0.501312, acc 0.72
2019-09-12T09:32:16.870338: step 142, loss 0.450857, acc 0.82
2019-09-12T09:32:17.050313: step 143, loss 0.42058, acc 0.8
2019-09-12T09:32:17.221355: step 144, loss 0.454145, acc 0.88
2019-09-12T09:32:17.405862: step 145, loss 0.50665, acc 0.78
2019-09-12T09:32:17.579398: step 146, loss 0.474781, acc 0.78
2019-09-12T09:32:17.755965: step 147, loss 0.353709, acc 0.86
2019-09-12T09:32:17.931993: step 148, loss 0.444757, acc 0.78
2019-09-1

2019-09-12T09:32:37.149391: step 265, loss 0.141557, acc 0.96
2019-09-12T09:32:37.282331: step 266, loss 0.203485, acc 0.942857
2019-09-12T09:32:37.450337: step 267, loss 0.159754, acc 0.94
2019-09-12T09:32:37.618325: step 268, loss 0.171577, acc 0.96
2019-09-12T09:32:37.788312: step 269, loss 0.243692, acc 0.92
2019-09-12T09:32:37.962248: step 270, loss 0.126554, acc 0.92
2019-09-12T09:32:38.134246: step 271, loss 0.161515, acc 0.94
2019-09-12T09:32:38.290169: step 272, loss 0.159705, acc 0.94
2019-09-12T09:32:38.450739: step 273, loss 0.250643, acc 0.94
2019-09-12T09:32:38.606169: step 274, loss 0.127158, acc 0.96
2019-09-12T09:32:38.771727: step 275, loss 0.154594, acc 0.94
2019-09-12T09:32:38.930599: step 276, loss 0.288426, acc 0.86
2019-09-12T09:32:39.100619: step 277, loss 0.211702, acc 0.9
2019-09-12T09:32:39.268825: step 278, loss 0.196654, acc 0.94
2019-09-12T09:32:39.435692: step 279, loss 0.134998, acc 0.96
2019-09-12T09:32:39.593894: step 280, loss 0.234547, acc 0.88
2019-

2019-09-12T09:32:58.475286: step 398, loss 0.273225, acc 0.84
2019-09-12T09:32:58.607309: step 399, loss 0.371944, acc 0.828571
2019-09-12T09:32:58.768270: step 400, loss 0.321306, acc 0.84

Evaluation:
2019-09-12T09:32:58.827344: step 400, loss 0.185445, acc 0.912621

2019-09-12T09:32:58.979252: step 401, loss 0.260883, acc 0.92
2019-09-12T09:32:59.143203: step 402, loss 0.183089, acc 0.96
2019-09-12T09:32:59.309199: step 403, loss 0.375111, acc 0.9
2019-09-12T09:32:59.455159: step 404, loss 0.141126, acc 0.96
2019-09-12T09:32:59.614167: step 405, loss 0.388869, acc 0.88
2019-09-12T09:32:59.771162: step 406, loss 0.17428, acc 0.94
2019-09-12T09:32:59.933159: step 407, loss 0.266648, acc 0.88
2019-09-12T09:33:00.103165: step 408, loss 0.212241, acc 0.9
2019-09-12T09:33:00.262148: step 409, loss 0.238464, acc 0.88
2019-09-12T09:33:00.412198: step 410, loss 0.384739, acc 0.88
2019-09-12T09:33:00.574224: step 411, loss 0.667145, acc 0.74
2019-09-12T09:33:00.726235: step 412, loss 0.213886

2019-09-12T09:33:19.272085: step 528, loss 0.0390104, acc 1
2019-09-12T09:33:19.425116: step 529, loss 0.0943075, acc 0.96
2019-09-12T09:33:19.590116: step 530, loss 0.153281, acc 0.94
2019-09-12T09:33:19.740122: step 531, loss 0.0945538, acc 0.96
2019-09-12T09:33:19.878135: step 532, loss 0.109299, acc 1
2019-09-12T09:33:20.032149: step 533, loss 0.131619, acc 0.96
2019-09-12T09:33:20.193200: step 534, loss 0.107767, acc 0.96
2019-09-12T09:33:20.349189: step 535, loss 0.0609726, acc 1
2019-09-12T09:33:20.513230: step 536, loss 0.0800296, acc 0.96
2019-09-12T09:33:20.670226: step 537, loss 0.0900319, acc 1
2019-09-12T09:33:20.837284: step 538, loss 0.0887122, acc 0.98
2019-09-12T09:33:20.985266: step 539, loss 0.0959681, acc 0.98
2019-09-12T09:33:21.145304: step 540, loss 0.0645877, acc 0.96
2019-09-12T09:33:21.294449: step 541, loss 0.0406863, acc 1
2019-09-12T09:33:21.459320: step 542, loss 0.047975, acc 1
2019-09-12T09:33:21.612313: step 543, loss 0.0840897, acc 0.98
2019-09-12T09:3

2019-09-12T09:33:41.011041: step 661, loss 0.0336956, acc 1
2019-09-12T09:33:41.168069: step 662, loss 0.060043, acc 0.98
2019-09-12T09:33:41.343069: step 663, loss 0.0865943, acc 0.98
2019-09-12T09:33:41.503045: step 664, loss 0.0267099, acc 1
2019-09-12T09:33:41.647087: step 665, loss 0.056185, acc 1
2019-09-12T09:33:41.809051: step 666, loss 0.031443, acc 1
2019-09-12T09:33:41.980056: step 667, loss 0.0448565, acc 0.98
2019-09-12T09:33:42.136554: step 668, loss 0.0480229, acc 1
2019-09-12T09:33:42.309053: step 669, loss 0.0560234, acc 0.98
2019-09-12T09:33:42.468056: step 670, loss 0.0225193, acc 1
2019-09-12T09:33:42.636530: step 671, loss 0.0262443, acc 1
2019-09-12T09:33:42.793021: step 672, loss 0.024322, acc 1
2019-09-12T09:33:42.962090: step 673, loss 0.0750209, acc 0.98
2019-09-12T09:33:43.119152: step 674, loss 0.0237946, acc 1
2019-09-12T09:33:43.291124: step 675, loss 0.0247665, acc 1
2019-09-12T09:33:43.447141: step 676, loss 0.081144, acc 0.98
2019-09-12T09:33:43.613111: